#### Importar

In [ ]:
# --- Análisis de Datos y Numérico ---
import pandas as pd
import numpy as np

# --- Visualización de Datos ---
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import font_manager
from adjustText import adjust_text

# Plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import plot

# --- Análisis de Redes y Grafos ---
import networkx as nx
from networkx import florentine_families_graph
from yfiles_jupyter_graphs import GraphWidget

# --- Utilidades Generales y de Jupyter ---
import colorsys
from random import random
from typing import Dict
from IPython.display import display


input_file_path = r"RUTA\obligaciones_combinadas_expandidas_ajustadas.xlsx"

df = pd.read_excel(input_file_path)

#### Corregir errores de Fuentes

In [ ]:
condicion = df['fuente'].str.contains('codigo de aguas', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto con fuerza de ley'
df.loc[condicion, 'numero_fuente_ajustado'] = 1122
df.loc[condicion, 'año_fuente'] = 1981

condicion = df['fuente'].str.contains('ordenanza', case=False, na=False) 
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 47
df.loc[condicion, 'año_fuente'] = 1992

condicion = df['numero_fuente_ajustado'].str.contains('urbanismo', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto con fuerza de ley'
df.loc[condicion, 'numero_fuente_ajustado'] = 458
df.loc[condicion, 'año_fuente'] = 1975

condicion = df['numero_fuente_ajustado'].str.contains('pesca', case=False, na=False)
df.loc[condicion, 'fuente'] = 'ley'
df.loc[condicion, 'numero_fuente_ajustado'] = 18892
df.loc[condicion, 'año_fuente'] = 1989

condicion = df['numero_fuente_ajustado'].str.contains('seguridad minera', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo' 
df.loc[condicion, 'numero_fuente_ajustado'] = 132
df.loc[condicion, 'año_fuente'] = 2002

condicion = df['numero_fuente_ajustado'].str.contains('residuos peligrosos', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 148
df.loc[condicion, 'año_fuente'] = 2003

condicion = df['numero_fuente_ajustado'].str.contains('bosques', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo' 
df.loc[condicion, 'numero_fuente_ajustado'] = 4363
df.loc[condicion, 'año_fuente'] = 1931

condicion = df['numero_fuente_ajustado'].str.contains('cargas', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 298
df.loc[condicion, 'año_fuente'] = 1994

condicion = df['numero_fuente_ajustado'].str.contains('corrientes fuertes', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] =  4188
df.loc[condicion, 'año_fuente'] = 1955

condicion = df['numero_fuente_ajustado'].str.contains('emisiones', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 1
df.loc[condicion, 'año_fuente'] = 2013

condicion = df['numero_fuente_ajustado'].str.contains('fuego', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 276
df.loc[condicion, 'año_fuente'] = 1980

condicion = df['numero_fuente_ajustado'].str.contains('transporte', case=False, na=False)
df.loc[condicion, 'fuente'] = 'decreto supremo'
df.loc[condicion, 'numero_fuente_ajustado'] = 75
df.loc[condicion, 'año_fuente'] = 1987

#### UN Solo N°PAS por cada RCA

In [ ]:
try:
    subset_cols = ['cell', 'numero_fuente_ajustado']
    mask_pas = (df['seccion'] == 'pas') & (df['numero_fuente_ajustado'] != "No identificado") & (df['numero_fuente_ajustado'].notna())
    
    duplicated_rows = df[mask_pas & df.duplicated(subset=subset_cols, keep=False)]

    if duplicated_rows.empty:
        print("No se encontraron filas duplicadas para modificar. El archivo de salida no fue creado.")
    else:
        print(f"Se encontraron {len(duplicated_rows)} filas involucradas en duplicados. Analizando grupos...\n")
        
        indices_to_modify = []
        duplicated_groups = duplicated_rows[subset_cols].drop_duplicates().values
        for cell, nfa in duplicated_groups:
            group_df = duplicated_rows[(duplicated_rows['cell'] == cell) & (duplicated_rows['numero_fuente_ajustado'] == nfa)]
            priority_condition = (group_df['numero_fuente'].astype(str) != group_df['numero_fuente_ajustado'].astype(str)) & group_df['numero_fuente'].notna()
            priority_rows = group_df[priority_condition]
            non_priority_rows = group_df[~priority_condition]

            kept_indices = []
            modified_indices_in_group = []
            reason = ""

            if not priority_rows.empty and not non_priority_rows.empty:
                reason = "Prioridad (numero_fuente != ajustado)"
                kept_indices.extend(priority_rows.index.tolist())
                modified_indices_in_group.extend(non_priority_rows.index.tolist())
                
            else:
                reason = "Mismo grupo (keep='first')"
                all_indices = group_df.index.tolist()
                kept_indices.append(all_indices[0])
                modified_indices_in_group.extend(all_indices[1:])

            indices_to_modify.extend(modified_indices_in_group)
            print(f"Grupo Duplicado: cell='{cell}', numero_fuente_ajustado='{nfa}'")
            print(f"  └─ Razón de la decisión: {reason}")
            kept_values = df.loc[kept_indices, 'numero_fuente_ajustado'].unique().tolist()
            modified_values = df.loc[modified_indices_in_group, 'numero_fuente_ajustado'].unique().tolist()

            if modified_indices_in_group:
                pass

        if indices_to_modify:
            print(f"\nSe modificarán un total de {len(indices_to_modify)} filas.")
            df.loc[indices_to_modify, 'numero_fuente_ajustado'] = 'No identificado - modificado'
        else:
             print("\nAunque se encontraron grupos duplicados, la lógica no requirió modificar ninguna fila.")


except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo en la ruta especificada:\n{input_file_path}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

In [ ]:
df.loc[df['seccion'] == 'pas', 'fuente'] = 'permisos ambientales sectoriales'

In [ ]:
extracted_years = df['numero_fuente_completo'].str.extract(r'\/(\d{4})$', expand=False).astype(float)
valid_years = extracted_years.where((extracted_years >= 1900) & (extracted_years <= 2025))
diferencia_absoluta = (df['año_fuente'] - valid_years).abs()
condicion_final = (valid_years.notna()) & ((diferencia_absoluta > 1) | (df['año_fuente'].isna()))
df.loc[condicion_final, 'año_fuente'] = valid_years[condicion_final]

### Corregir Duplicadas y las con más de 1 fuente

In [ ]:
fuentes_a_limpiar = ["Compromiso ambiental voluntario", "decisión de autoridad"]
df.loc[df['fuente'].isin(fuentes_a_limpiar), 'numero_fuente_ajustado'] = ""
df.loc[df['año_fuente'].isin(fuentes_a_limpiar), 'numero_fuente_ajustado'] = ""

In [ ]:
df = df.drop_duplicates(subset=['obligacion_id', 'fuente', 'numero_fuente_ajustado'], keep='first')
def aplicar_logica_filtrado(grupo):
    if len(grupo) <= 1:
        return grupo

    fuentes_presentes = set(grupo['fuente'])
    if "permisos ambientales sectoriales" in fuentes_presentes:
        return grupo[grupo['fuente'] == "permisos ambientales sectoriales"]

    if "Compromiso ambiental voluntario" in fuentes_presentes and "decisión de autoridad" in fuentes_presentes:
        return grupo[grupo['fuente'] == "Compromiso ambiental voluntario"]

    if "decisión de autoridad" in fuentes_presentes:
        otras_fuentes = fuentes_presentes - {"decisión de autoridad", "Compromiso ambiental voluntario", "permisos ambientales sectoriales"}
        if otras_fuentes:
            return grupo[grupo['fuente'] != "decisión de autoridad"]

    return grupo

df = df.groupby('obligacion_id', group_keys=False).apply(aplicar_logica_filtrado)
df = df.reset_index(drop=True)

### Corregir Otros

In [ ]:
def aplicar_correcciones_normativas(df):
    """
    Aplica correcciones masivas a los datos normativos según tabla de conversiones
    Versión mejorada que maneja diferentes tipos de datos y formatos
    """
    
    def normalizar_fuente(fuente):
        """Normaliza el texto de la fuente para comparaciones consistentes"""
        if pd.isna(fuente):
            return ''
        return str(fuente).lower().strip()
    
    def normalizar_numero(numero):
        """Normaliza el número para comparaciones consistentes"""
        if pd.isna(numero):
            return None
        try:
            # Convertir a int para comparación consistente
            return int(float(str(numero).strip()))
        except (ValueError, TypeError):
            return None
    
    def normalizar_año(año):
        """Normaliza el año para comparaciones consistentes"""
        if pd.isna(año):
            return None
        try:
            # Convertir a int para comparación consistente
            return int(float(str(año).strip()))
        except (ValueError, TypeError):
            return None
    
    # Tabla de correcciones: (fuente_origen, numero_origen, año_origen) -> (fuente_destino, numero_destino, año_destino)
    correcciones = [
        # (fuente_origen, numero_origen, año_origen, fuente_destino, numero_destino, año_destino)
        ('ley', 17798, 1972, 'decreto supremo', 400, 1977),
        ('decreto supremo', 400, 1972, 'decreto supremo', 400, 1977),
        ('decreto supremo', 149, 2006, 'decreto supremo', 80, 2004),
        ('decreto con fuerza de ley', 4, 1994, 'decreto supremo', 4, 1994), 
        ('decreto supremo', 68, 2021, 'decreto supremo', 327, 1997),
        ('decreto supremo', 327, 2021, 'decreto supremo', 327, 1997),
        ('decreto supremo', 31, 2017, 'decreto supremo', 31, 2016),
        ('decreto supremo', 23, 1926, 'decreto supremo', 236, 1926),
        ('decreto supremo', 157, 2007, 'decreto supremo', 157, 2005),
        ('decreto supremo', 72, 1985, 'decreto supremo', 132, 2002),
        ('decreto supremo', 132, 1985, 'decreto supremo', 132, 2002),
        ('decreto supremo', 1150, 1980, 'decreto supremo', 100, 2005),
        ('decreto supremo', 100, 1980, 'decreto supremo', 100, 2005),
        ('decreto ley', 93557, 1980, 'decreto ley', 3557, 1980),
        ('decreto ley', 701, 1974, 'decreto ley', 2565, 1979),
        ('decreto ley', 701, 1979, 'decreto ley', 2565, 1979),
        ('ley', 15840, 1964, 'decreto con fuerza de ley', 850, 1997),
        ('decreto supremo', 294, 1984, 'decreto con fuerza de ley', 850, 1997),
        ('decreto con fuerza de ley', 850, 1984, 'decreto con fuerza de ley', 850, 1997),
        ('decreto con fuerza de ley', 850, 1964, 'decreto con fuerza de ley', 850, 1997),
        ('ley', 20724, 2014, 'decreto con fuerza de ley', 725, 1967),
        ('decreto supremo', 90, 2010, 'decreto con fuerza de ley', 725, 1967),
        ('decreto supremo', 90, 2011, 'decreto con fuerza de ley', 725, 1967),
        ('ley', 20443, 2010, 'decreto con fuerza de ley', 458, 1975),
        ('decreto con fuerza de ley', 1, 1982, 'decreto con fuerza de ley', 4, 2006),
        ('decreto con fuerza de ley', 4, 1982, 'decreto con fuerza de ley', 4, 2006),
        ('decreto con fuerza de ley', 4, 2007, 'decreto con fuerza de ley', 4, 2006), 
        ('decreto con fuerza de ley', 1, 2009, 'decreto con fuerza de ley', 1, 2007), 
        ('ley', 18290, 1984, 'decreto con fuerza de ley', 1, 2007),
        ('ley', 18892, 1989, 'decreto supremo', 430, 1991), 
        ('decreto supremo', 938, 2011, 'decreto supremo', 38, 2011), 
        ('ley', 34, 2020, 'decreto supremo', 160, 2008),
        ('decreto supremo', 34, 2020, 'decreto supremo', 160, 2008),
        ('decreto con fuerza de ley', 34, 2020, 'decreto supremo', 160, 2008),
    ]
    
    print(f"Aplicando {len(correcciones)} correcciones...")
    print("Normalizando datos del DataFrame...")
    
    df_normalizado = df.copy()
    df_normalizado['fuente_norm'] = df['fuente'].apply(normalizar_fuente)
    df_normalizado['numero_norm'] = df['numero_fuente_ajustado'].apply(normalizar_numero)
    df_normalizado['año_norm'] = df['año_fuente'].apply(normalizar_año)
    registros_modificados = 0
    for correccion in correcciones:
        fuente_orig, numero_orig, año_orig, fuente_dest, numero_dest, año_dest = correccion
        fuente_orig_norm = normalizar_fuente(fuente_orig)
        numero_orig_norm = normalizar_numero(numero_orig)
        año_orig_norm = normalizar_año(año_orig)
        

        condicion = (
            (df_normalizado['fuente_norm'] == fuente_orig_norm) & 
            (df_normalizado['numero_norm'] == numero_orig_norm) & 
            (df_normalizado['año_norm'] == año_orig_norm)
        )
        registros_coincidentes = condicion.sum()
        
        if registros_coincidentes > 0:
            df.loc[condicion, 'fuente'] = fuente_dest
            df.loc[condicion, 'numero_fuente_ajustado'] = numero_dest
            df.loc[condicion, 'año_fuente'] = año_dest
            
            registros_modificados += registros_coincidentes
            print(f"✓ Corregidos {registros_coincidentes} registros: "
                  f"{fuente_orig} {numero_orig}/{año_orig} → "
                  f"{fuente_dest} {numero_dest}/{año_dest}")
        else:
            print(f"⚠ No se encontraron registros para: "
                  f"{fuente_orig} {numero_orig}/{año_orig}")
    
    print(f"\nResumen: Se modificaron {registros_modificados} registros en total.")
    return df

df = aplicar_correcciones_normativas(df)

### Corrección Automática a Números Altos (que dificilmente serían una coincidencia)

In [ ]:
def aplicar_correcciones_por_numero(df):
    """
    Aplica correcciones basándose ÚNICAMENTE en el numero_fuente_ajustado
    Si encuentra el número, reemplaza toda la información con los datos correctos
    """
    
    def normalizar_numero(numero):
        """Normaliza el número para comparaciones consistentes"""
        if pd.isna(numero):
            return None
        try:
            return int(float(str(numero).strip()))
        except (ValueError, TypeError):
            return None
    
    correcciones_por_numero = {
        160: ('decreto supremo', 2008),
        193: ('decreto supremo', 1998),
        194: ('decreto supremo', 1973),
        200: ('decreto supremo', 1993),
        211: ('decreto supremo', 1991),
        232: ('resolucion', 2002),
        236: ('decreto supremo', 1926),
        244: ('decreto supremo', 2005),
        248: ('decreto supremo', 2007),
        259: ('decreto supremo', 1980),
        276: ('decreto supremo', 1980),
        279: ('decreto supremo', 1983),
        291: ('decreto supremo', 2007),
        298: ('decreto supremo', 1994),
        300: ('decreto supremo', 1994),
        327: ('decreto supremo', 1997),
        400: ('decreto supremo', 1977),
        405: ('decreto supremo', 1983),
        430: ('decreto supremo', 1991),
        458: ('decreto con fuerza de ley', 1975),
        461: ('decreto supremo', 1995),
        484: ('decreto supremo', 1990),
        531: ('decreto supremo', 1967),
        594: ('decreto supremo', 1999),
        655: ('decreto supremo', 1940),
        725: ('decreto con fuerza de ley', 1967),
        735: ('decreto supremo', 1969),
        830: ('decreto ley', 1974),
        850: ('decreto con fuerza de ley', 1997),
        867: ('decreto supremo', 1978),
        1122: ('decreto con fuerza de ley', 1981),
        1164: ('decreto supremo', 1974),
        1215: ('ley', 1978),
        1261: ('decreto supremo', 1957),
        499: ('resolucion', 2006),
        1665: ('decreto supremo', 2002),
        2565: ('decreto ley', 1979),
        3557: ('decreto ley', 1980),
        4188: ('decreto supremo', 1955),
        4363: ('decreto supremo', 1931),
        4601: ('decreto supremo', 1929),
        16744: ('ley', 1968),
        17288: ('ley', 1970),
        18248: ('ley', 1983),
        18378: ('ley', 1984),
        18695: ('ley', 1988),
        18755: ('ley', 1989),
        18834: ('ley', 1989),
        19253: ('ley', 1993),
        19300: ('ley', 1994),
        19473: ('ley', 1996),
        19880: ('ley', 2003),
        20001: ('ley', 2005),
        20096: ('ley', 2006),
        20283: ('ley', 2008),
        20380: ('ley', 2009),
        20389: ('ley', 2009),
        20417: ('ley', 2010),
        20551: ('ley', 2011),
        20879: ('ley', 2015),
        20920: ('ley', 2016),
        20936: ('ley', 2017),
        21455: ('ley', 2022),
        1139: ('resolucion', 2013),
        1518: ('resolucion', 2013),
        172: ('decreto supremo', 1988),
        223: ('resolucion', 2015),
        359: ('resolucion', 2005),
        446: ('decreto supremo', 2006),
        610: ('resolucion', 1982),
        878: ('decreto supremo', 2011)
    }
    
    print(f"Aplicando correcciones por número para {len(correcciones_por_numero)} números diferentes...")
    
    df['numero_norm_temp'] = df['numero_fuente_ajustado'].apply(normalizar_numero)
    
    registros_modificados = 0
    numeros_encontrados = []
    
    for numero, (fuente_correcta, año_correcto) in correcciones_por_numero.items():
        condicion = df['numero_norm_temp'] == numero
        registros_coincidentes = condicion.sum()
        
        if registros_coincidentes > 0:
            registros_actuales = df[condicion][['fuente', 'numero_fuente_ajustado', 'año_fuente']].drop_duplicates()
            
            print(f"✓ Número {numero} encontrado en {registros_coincidentes} registros:")
            for _, registro in registros_actuales.iterrows():
                print(f"  {registro['fuente']} {registro['numero_fuente_ajustado']}/{registro['año_fuente']} → {fuente_correcta} {numero}/{año_correcto}")
            df.loc[condicion, 'fuente'] = fuente_correcta
            df.loc[condicion, 'numero_fuente_ajustado'] = numero
            df.loc[condicion, 'año_fuente'] = año_correcto
            
            registros_modificados += registros_coincidentes
            numeros_encontrados.append(numero)
    
    df.drop('numero_norm_temp', axis=1, inplace=True)
    
    numeros_no_encontrados = set(correcciones_por_numero.keys()) - set(numeros_encontrados)
    
    print(f"\nResumen:")
    print(f"- Se modificaron {registros_modificados} registros en total")
    print(f"- Se encontraron {len(numeros_encontrados)} números diferentes en los datos")
    
    if numeros_no_encontrados:
        print(f"- Números no encontrados en los datos: {sorted(list(numeros_no_encontrados))[:10]}{'...' if len(numeros_no_encontrados) > 10 else ''}")
    
    return df

df = aplicar_correcciones_por_numero(df)

In [ ]:
output_file_path = r"RUTA\obligaciones_combinadas_expandidas_corregidas.xlsx"
df.to_excel(output_file_path, index=False)

#### Comprimir

In [ ]:
def comprimir_fuentes(df_expandido):
    """Comprime el DataFrame agrupando por 'obligacion_id'."""
    columnas_a_agregar = {
        col: 'first' for col in df_expandido.columns
        if col not in ['obligacion_id', 'fuente', 'numero_fuente_ajustado', 'año_fuente']
    }
    columnas_a_agregar['fuente'] = lambda x: list(x)
    columnas_a_agregar['numero_fuente_ajustado'] = lambda x: list(x)
    columnas_a_agregar['año_fuente'] = lambda x: list(x)
    df_comprimido = df_expandido.groupby('obligacion_id').agg(columnas_a_agregar).reset_index()
    return df_comprimido

df['fuente'] = df['fuente'].str.replace('Compromiso ambiental voluntario', 'compromiso ambiental voluntario')
df_comprimido = comprimir_fuentes(df)
def extraer_pas_y_limpiar(row):
    """
    Busca la fuente 'Permisos Ambientales Sectoriales', extrae su número a una nueva
    columna 'pas' y devuelve las listas limpias sin esa fuente.
    """
    fuentes = row['fuente']
    numeros = row['numero_fuente_ajustado']
    años = row['año_fuente']
    
    pas_valor = np.nan
    fuentes_limpias = []
    numeros_limpios = []
    años_limpios = []
    for i, fuente in enumerate(fuentes):
        if fuente == 'permisos ambientales sectoriales':
            pas_valor = numeros[i] 
        else:
            fuentes_limpias.append(fuentes[i])
            numeros_limpios.append(numeros[i])
            años_limpios.append(años[i])
            
    return pas_valor, fuentes_limpias, numeros_limpios, años_limpios
    
# Asignamos los resultados a nuevas columnas en el DataFrame
df_comprimido[['pas', 'fuente', 'numero_fuente_ajustado', 'año_fuente']] = df_comprimido.apply(extraer_pas_y_limpiar, axis=1, result_type='expand')

In [ ]:
max_fuentes = df_comprimido['fuente'].str.len().max()
for i in range(max_fuentes):
    fuente_i = df_comprimido['fuente'].str[i]
    numero_i = df_comprimido['numero_fuente_ajustado'].str[i]
    año_i = df_comprimido['año_fuente'].str[i]

    numeros_validos = pd.to_numeric(numero_i, errors='coerce')
    años_validos = pd.to_numeric(año_i, errors='coerce')

    texto_fuente = fuente_i.fillna('')
    texto_numero = numeros_validos.dropna().astype(int).astype(str)
    texto_año = años_validos.dropna().astype(int).astype(str)

    texto_base = texto_fuente.add(' ' + texto_numero, fill_value='').str.strip()
    columna_final = texto_base.add('/' + texto_año, fill_value='').str.strip()

    df_comprimido[f'fuente_{i+1}'] = columna_final.replace('', np.nan)

columnas_a_borrar = ['numero_fuente_ajustado', 'año_fuente', 'fuente_5', 'fuente_6', 'fuente_7', 'fuente_8', 'fuente_9', 'fuente_10']
df_comprimido = df_comprimido.drop(columns=columnas_a_borrar)

In [ ]:
condicion = (df_comprimido['seccion'] == 'pas') & \
            (df_comprimido['fuente_1'].notna()) & \
            (df_comprimido['pas'].isna()) & \
            (df_comprimido['numero_fuente'].notna())

df_comprimido.loc[condicion, 'pas'] = df_comprimido.loc[condicion, 'numero_fuente']
df_comprimido.loc[condicion, 'fuente_1'] = np.nan
df_comprimido.loc[condicion, 'fuente_2'] = np.nan
df_comprimido.loc[condicion, 'fuente_3'] = np.nan
df_comprimido.loc[condicion, 'fuente_4'] = np.nan

In [ ]:
output_file_path = r"RUTA\obligaciones_combinadas_corregidas.xlsx"
df_comprimido.to_excel(output_file_path, index=False)

### GRAFOS

In [ ]:
archivo_excel = r"RUTA\normativa.xlsx"
font_path = r"RUTA\fuente\gobCL_Bold.ttf"
try:
    font_manager.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'gobCL'
    print("Fuente gobCL cargada exitosamente")
except Exception as e:
    print(f"No se pudo cargar la fuente gobCL: {e}")
    print("Usando fuente por defecto")

try:
    pareadas = pd.read_excel(archivo_excel, sheet_name="Pareadas")
    print("Datos de 'Pareadas' cargados:")
    print(pareadas.describe())
    ponderacion = pd.read_excel(archivo_excel, sheet_name="Ponderación")
    print("\nDatos de 'Ponderación' cargados:")
    print(ponderacion.head())

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada:\n{archivo_excel}")
    exit()

edge_list = pareadas.rename(columns={ 'fuente_1': 'P1', 'fuente_2': 'P2','Valor': 'weight'}).copy()
relaciones_reales = edge_list[edge_list['P1'] != edge_list['P2']]

print(f"\nAnálisis de datos:")
print(f"Total de filas: {len(edge_list)}")
print(f"Auto-loops (fuente consigo misma): {len(edge_list) - len(relaciones_reales)}")
print(f"Relaciones entre diferentes fuentes: {len(relaciones_reales)}")

if not relaciones_reales.empty:
    print("¡Se encontraron relaciones reales entre fuentes! Usando estos datos.")
    edge_list = relaciones_reales
else:
    print("Advertencia: Solo hay auto-loops. No se pueden construir conexiones significativas.")
    edge_list = pd.DataFrame(columns=['P1', 'P2', 'weight'])

if not edge_list.empty:
    min_w = edge_list['weight'].min()
    max_w = edge_list['weight'].max()

    if max_w > min_w:
        edge_list['weight_normalized'] = (edge_list['weight'] - min_w) / (max_w - min_w)
    else:
        edge_list['weight_normalized'] = 0.5

    edge_list['layout_weight'] = (edge_list['weight_normalized'] * 9) + 1  
    edge_list["inv_weight"] = 1.01 - edge_list['weight_normalized']
    min_width, max_width = 1, 8
    edge_list['line_width'] = min_width + (edge_list['weight_normalized'] * (max_width - min_width))
    
    print("\nEdge list con atributos de visualización:")
    print(edge_list[['P1', 'P2', 'weight_normalized', 'layout_weight', 'line_width']].head())
else:
    print("\nNo hay aristas válidas para construir la red.")
    exit()



g = nx.from_pandas_edgelist(edge_list, source="P1", target="P2", 
                           edge_attr=["weight_normalized", "inv_weight", "layout_weight", "line_width"])
print(f"\nRed completa - Nodos: {g.number_of_nodes()}, Aristas: {g.number_of_edges()}")


mst = nx.minimum_spanning_tree(g, weight='inv_weight')
print(f"MST - Nodos: {mst.number_of_nodes()}, Aristas: {mst.number_of_edges()}")
ps = nx.Graph()

# 1. Añadir todas las aristas del MST
for u, v, data in mst.edges(data=True):
    ps.add_edge(u, v, 
                weight_normalized=data.get("weight_normalized", 0.5),
                layout_weight=data.get("layout_weight", 5),
                line_width=data.get("line_width", 3))

# 2. Añadir aristas adicionales que superen un umbral de similitud
umbral = 0.017544 ## Percentil 25
for u, v, data in g.edges(data=True):
    if data['weight_normalized'] >= umbral:
        ps.add_edge(u, v, 
                    weight_normalized=data.get("weight_normalized", 0.5),
                    layout_weight=data.get("layout_weight", 5),
                    line_width=data.get("line_width", 3))

print(f"\nRed final (ps) - Nodos: {ps.number_of_nodes()}, Aristas: {ps.number_of_edges()}")

node_list = list(ps.nodes)
edge_list = list(ps.edges)
positions = nx.spring_layout(ps)
print(positions)

fig = plt.figure(figsize=(50,20))
ax = fig.gca()
nx.draw_networkx(ps, ax=ax, node_size=750, width=1)

plt.savefig("productspace1.png")

ponderacion.drop_duplicates(subset=['fuente'], keep='first', inplace=True)
ponderacion_dict = ponderacion.set_index('fuente').to_dict('index')
nodos_a_eliminar = []
for node_id in ps.nodes():
    if node_id in ponderacion_dict:
        ministerio = ponderacion_dict[node_id].get('ministerio', 'Sin clasificar')
        if ministerio == 'Sin clasificar':
            nodos_a_eliminar.append(node_id)
    else:
        nodos_a_eliminar.append(node_id)

# Eliminar nodos "Sin clasificar"
ps.remove_nodes_from(nodos_a_eliminar)
print(f"Eliminados {len(nodos_a_eliminar)} nodos 'Sin clasificar'")
print(f"Red final después de filtrado - Nodos: {ps.number_of_nodes()}, Aristas: {ps.number_of_edges()}")
for node_id in ps.nodes():
    if node_id in ponderacion_dict:
        nx.set_node_attributes(ps, {node_id: ponderacion_dict[node_id]})

ultra_orange_colors = [
    "#838D9B", "#D2610C", "#F4D25A", "#EB8A2D", "#A03B26", "#9c6111", 
    "#bab106", "#FB8281", "#C42424", "#2b478a", "#6A9FB0", "#7B4F71",
]

ministerios_unicos = ponderacion['ministerio'].unique()
color_map_ministerios = {
    ministerio: ultra_orange_colors[i % len(ultra_orange_colors)]
    for i, ministerio in enumerate(ministerios_unicos)
}

# Calcular rangos para el tamaño de nodos
ponderaciones = [data.get('ponderación', 1) for node, data in ps.nodes(data=True)]
min_ponderacion = min(ponderaciones) if ponderaciones else 1
max_ponderacion = max(ponderaciones) if ponderaciones else 1


if ps.number_of_nodes() > 0:
    plt.figure(figsize=(22, 18))

    pos = nx.spring_layout(ps, 
                          weight='layout_weight', 
                          k=1.0/np.sqrt(ps.number_of_nodes()),  
                          iterations=2000,  
                          seed=42)

    # Preparar colores de nodos
    node_colors = [color_map_ministerios.get(ps.nodes[node].get('ministerio', 'Sin clasificar')) 
                   for node in ps.nodes()]
    node_sizes = []
    for node in ps.nodes():
        pond = ps.nodes[node].get('ponderación', min_ponderacion)
        if max_ponderacion > min_ponderacion:
            normalized = (pond - min_ponderacion) / (max_ponderacion - min_ponderacion)
            size = 300 + normalized * 5500
        else:
            size = 500
        node_sizes.append(size)

    edge_widths = [data['line_width'] for u, v, data in ps.edges(data=True)]
    edge_colors = []
    for u, v, data in ps.edges(data=True):
        intensity = data['weight_normalized']
        color_intensity = 0.7 
        edge_colors.append((color_intensity, color_intensity, color_intensity))

    nx.draw_networkx_edges(ps, pos, 
                          width=edge_widths,
                          edge_color=edge_colors,
                          alpha=0.8)


    node_data = [(node, size) for node, size in zip(ps.nodes(), node_sizes)]
    node_data.sort(key=lambda x: x[1])  
    
    for node, size in node_data:
        node_idx = list(ps.nodes()).index(node)
        nx.draw_networkx_nodes(ps, pos, 
                              nodelist=[node],
                              node_color=[node_colors[node_idx]], 
                              node_size=[size],
                              alpha=1.0,
                              linewidths=0.7, 
                              edgecolors='#2C2C2C')  

    def texts_overlap(text1, text2, threshold=0.1):
        """Detecta si dos textos se solapan significativamente"""
        bbox1 = text1.get_window_extent()
        bbox2 = text2.get_window_extent()
        
        # Convertir a coordenadas de datos
        bbox1_data = bbox1.transformed(plt.gca().transData.inverted())
        bbox2_data = bbox2.transformed(plt.gca().transData.inverted())
        
        x1_min, y1_min = bbox1_data.x0, bbox1_data.y0
        x1_max, y1_max = bbox1_data.x1, bbox1_data.y1
        x2_min, y2_min = bbox2_data.x0, bbox2_data.y0
        x2_max, y2_max = bbox2_data.x1, bbox2_data.y1
        
        if (x1_max > x2_min and x1_min < x2_max and 
            y1_max > y2_min and y1_min < y2_max):
            
            intersect_width = min(x1_max, x2_max) - max(x1_min, x2_min)
            intersect_height = min(y1_max, y2_max) - max(y1_min, y2_min)
            intersect_area = intersect_width * intersect_height
            
            area1 = (x1_max - x1_min) * (y1_max - y1_min)
            area2 = (x2_max - x2_min) * (y2_max - y2_min)
            min_area = min(area1, area2)
            
            return intersect_area / min_area > threshold
        
        return False


    # Crear todas las etiquetas centradas inicialmente
    texts = []
    for i, node in enumerate(ps.nodes()):
        # Reemplazar espacios por saltos de línea
        label_multilinea = str(node).replace(' ', '\n')
        x, y = pos[node]
        
        node_size = node_sizes[i]
        min_font_size, max_font_size = 11, 20
        min_node_size, max_node_size = min(node_sizes), max(node_sizes)
        
        if max_node_size > min_node_size:
            normalized_size = (node_size - min_node_size) / (max_node_size - min_node_size)
            font_size = min_font_size + (normalized_size * (max_font_size - min_font_size))
        else:
            font_size = 12
        
        text = plt.text(x, y, label_multilinea, 
                       fontsize=font_size, 
                       fontweight='bold', 
                       color='#1C1C1C',
                       family='gobCL',
                       ha='center', 
                       va='center')
        texts.append(text)

    # Forzar renderizado para obtener dimensiones correctas
    plt.gcf().canvas.draw()

    overlapping_texts = []
    for i, text1 in enumerate(texts):
        for j, text2 in enumerate(texts[i+1:], i+1):
            if texts_overlap(text1, text2, threshold=0.2):  
                if text1 not in overlapping_texts:
                    overlapping_texts.append(text1)
                if text2 not in overlapping_texts:
                    overlapping_texts.append(text2)

    print(f"Etiquetas con solapamiento detectado: {len(overlapping_texts)} de {len(texts)}")

    
    # Ordenar ministerios para que "Otros" aparezca al final
    ministerios_ordenados = sorted(color_map_ministerios.items(), 
                                  key=lambda x: (x[0] == 'Otros', x[0]))
    
    legend_elements = [Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=color, markersize=16, label=ministerio,
                             markeredgecolor='#1C1C1C', markeredgewidth=1.5)
                      for ministerio, color in ministerios_ordenados]
    
    legend = plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1.02, 0.5),
                       frameon=True, fancybox=True, shadow=True, 
                       fontsize=16, title='Ministerio', title_fontsize=18)
    
    # Configurar estilo de la leyenda
    legend.get_frame().set_facecolor('#FFFFFF')
    legend.get_frame().set_edgecolor('#1C1C1C')
    legend.get_frame().set_linewidth(1.5)
    
    for text in legend.get_texts():
        text.set_fontfamily('gobCL')
        text.set_color('#1C1C1C')
    
    legend.get_title().set_fontfamily('gobCL')
    legend.get_title().set_color('#1C1C1C')
    legend.get_title().set_fontweight('bold')
    
    plt.gcf().patch.set_facecolor('#FFFFFF')
    plt.gca().set_facecolor('#FFFFFF')
    
    plt.tight_layout(pad=0.5)
    plt.subplots_adjust(left=0.02, right=0.85, top=0.95, bottom=0.02)
    plt.axis('off')
    plt.savefig("red_fuentes_normativas_ultra_orange.pdf", dpi=300, bbox_inches='tight', 
               facecolor='#FFFFFF', edgecolor='none', pad_inches=0.1)
    plt.show()

    print("\n¡Gráfico con estilo Ultra Orange guardado como 'red_fuentes_normativas_ultra_orange.pdf'!")
    
    weights = [data['weight_normalized'] for u, v, data in ps.edges(data=True)]
    widths = [data['line_width'] for u, v, data in ps.edges(data=True)]
    
    print(f"\n=== ESTADÍSTICAS DE ARISTAS ===")
    print(f"Peso promedio (normalizado): {np.mean(weights):.3f}")
    print(f"Peso máximo: {np.max(weights):.3f}")
    print(f"Peso mínimo: {np.min(weights):.3f}")
    print(f"Ancho de línea promedio: {np.mean(widths):.1f}")
    print(f"Ancho máximo: {np.max(widths):.1f}")
    print(f"Ancho mínimo: {np.min(widths):.1f}")
    
else:
    print("\nNo se puede crear la visualización: la red no tiene nodos.")

print(f"\n=== ESTADÍSTICAS DE LA RED ===")
if ps.number_of_nodes() > 0:
    print(f"Número de nodos: {ps.number_of_nodes()}")
    print(f"Número de aristas: {ps.number_of_edges()}")
    print(f"Densidad de la red: {nx.density(ps):.4f}")

    if nx.is_connected(ps):
        print("La red está completamente conectada.")
    else:
        print(f"La red no está conectada. Componentes: {nx.number_connected_components(ps)}")

    centralidad = nx.degree_centrality(ps)
    top_centrales = sorted(centralidad.items(), key=lambda item: item[1], reverse=True)[:5]

    print(f"\n--- TOP 5 FUENTES MÁS CENTRALES ---")
    for fuente, cent in top_centrales:
        ministerio = ps.nodes[fuente].get('ministerio', 'N/A')
        print(f"- {fuente} (Ministerio: {ministerio}, Centralidad: {cent:.3f})")
        
    edge_weights = [(u, v, data['weight_normalized']) for u, v, data in ps.edges(data=True)]
    top_edges = sorted(edge_weights, key=lambda x: x[2], reverse=True)[:5]
    
    print(f"\n--- TOP 5 CONEXIONES MÁS FUERTES ---")
    for u, v, weight in top_edges:
        print(f"- {u} ↔ {v} (Similitud: {weight:.3f})")

In [ ]:
print(f"\n=== EXPORTANDO ARISTAS A EXCEL ===")

if ps.number_of_nodes() > 0 and ps.number_of_edges() > 0:
    # Crear lista para almacenar las aristas
    aristas_finales = []
    
    # Extraer todas las aristas de la red final con sus atributos
    for u, v, data in ps.edges(data=True):
        valor_original = None
        for _, row in pareadas.iterrows():
            if ((row['fuente_1'] == u and row['fuente_2'] == v) or 
                (row['fuente_1'] == v and row['fuente_2'] == u)):
                valor_original = row['Valor']
                break
        
        arista = {
            'fuente_1': u,
            'fuente_2': v,
            'ponderacion': valor_original if valor_original is not None else 0,  
            'peso_normalizado': data.get('weight_normalized', 0),  
            'ancho_linea': data.get('line_width', 1), 
            'peso_layout': data.get('layout_weight', 1)  
        }
        aristas_finales.append(arista)
    
    # Convertir a DataFrame
    df_aristas = pd.DataFrame(aristas_finales)
    df_aristas = df_aristas.sort_values('ponderacion', ascending=False)
    df_aristas['ministerio_fuente_1'] = df_aristas['fuente_1'].map(
        lambda x: ps.nodes[x].get('ministerio', 'N/A') if x in ps.nodes else 'N/A'
    )
    df_aristas['ministerio_fuente_2'] = df_aristas['fuente_2'].map(
        lambda x: ps.nodes[x].get('ministerio', 'N/A') if x in ps.nodes else 'N/A'
    )
    df_aristas['ponderacion_nodo_1'] = df_aristas['fuente_1'].map(
        lambda x: ponderacion_dict[x].get('ponderación', 0) if x in ponderacion_dict else 0
    )
    df_aristas['ponderacion_nodo_2'] = df_aristas['fuente_2'].map(
        lambda x: ponderacion_dict[x].get('ponderación', 0) if x in ponderacion_dict else 0
    )
    directorio_salida = os.path.dirname(archivo_excel)
    archivo_salida = os.path.join(directorio_salida, "aristas_red_final.xlsx")
    try:
        with pd.ExcelWriter(archivo_salida, engine='openpyxl') as writer:
            df_aristas.to_excel(writer, sheet_name='Aristas_Red_Final', index=False)
            estadisticas = {
                'Métrica': [
                    'Número total de aristas',
                    'Número de nodos conectados',
                    'Ponderación promedio',
                    'Ponderación máxima',
                    'Ponderación mínima',
                    'Densidad de la red',
                    'Red conectada'
                ],
                'Valor': [
                    ps.number_of_edges(),
                    ps.number_of_nodes(),
                    df_aristas['ponderacion'].mean(),
                    df_aristas['ponderacion'].max(),
                    df_aristas['ponderacion'].min(),
                    nx.density(ps),
                    'Sí' if nx.is_connected(ps) else 'No'
                ]
            }
            
            df_estadisticas = pd.DataFrame(estadisticas)
            df_estadisticas.to_excel(writer, sheet_name='Estadisticas', index=False)
            
            # Hoja con resumen por ministerio
            ministerios_conectados = set()
            for _, row in df_aristas.iterrows():
                ministerios_conectados.add(row['ministerio_fuente_1'])
                ministerios_conectados.add(row['ministerio_fuente_2'])
            
            resumen_ministerios = []
            for ministerio in ministerios_conectados:
                if ministerio != 'N/A':
                    # Contar conexiones donde participa este ministerio
                    conexiones = len(df_aristas[
                        (df_aristas['ministerio_fuente_1'] == ministerio) | 
                        (df_aristas['ministerio_fuente_2'] == ministerio)
                    ])
                    
                    # Promedio de ponderación
                    ponderaciones_ministerio = df_aristas[
                        (df_aristas['ministerio_fuente_1'] == ministerio) | 
                        (df_aristas['ministerio_fuente_2'] == ministerio)
                    ]['ponderacion']
                    
                    resumen_ministerios.append({
                        'Ministerio': ministerio,
                        'Número_Conexiones': conexiones,
                        'Ponderación_Promedio': ponderaciones_ministerio.mean(),
                        'Ponderación_Máxima': ponderaciones_ministerio.max()
                    })
            
            df_ministerios = pd.DataFrame(resumen_ministerios)
            df_ministerios = df_ministerios.sort_values('Número_Conexiones', ascending=False)
            df_ministerios.to_excel(writer, sheet_name='Resumen_Ministerios', index=False)
        
        print(f"✅ Archivo Excel creado exitosamente:")
        print(f"📄 Ruta: {archivo_salida}")
        print(f"📊 Total de aristas exportadas: {len(df_aristas)}")
        print(f"📈 Ponderación promedio: {df_aristas['ponderacion'].mean():.4f}")
        
        # Mostrar las top 10 conexiones más fuertes
        print(f"\n--- TOP 10 CONEXIONES MÁS FUERTES (VALORES ORIGINALES) ---")
        top_10 = df_aristas.head(10)
        for idx, row in top_10.iterrows():
            print(f"• {row['fuente_1']} ↔ {row['fuente_2']} (Valor original: {row['ponderacion']:.4f})")
            
    except Exception as e:
        print(f"❌ Error al guardar el archivo Excel: {e}")
        
else:
    print("❌ No se pueden exportar las aristas: la red no tiene conexiones válidas.")

### Otras

In [ ]:
# --- PASO EXTRA: ANÁLISIS DE EXCLUSIONES ---
print("\n=== ANÁLISIS DE ELEMENTOS EXCLUIDOS ===")

# La lista 'nodos_a_eliminar' ya fue creada en el Paso 7.
print(f"\n--- Nodos Excluidos ---")
print(f"Total de nodos excluidos (ej. 'Sin clasificar'): {len(nodos_a_eliminar)}")
if len(nodos_a_eliminar) > 0:
    print("Algunos nodos excluidos:")
    print(nodos_a_eliminar[:10]) # Imprime los primeros 10 para no saturar la consola

# Comparamos las aristas del grafo completo 'g' con las del grafo final 'ps'
aristas_totales = set(g.edges())
aristas_finales = set(ps.edges())

aristas_excluidas = aristas_totales - aristas_finales

print(f"\n--- Aristas Excluidas ---")
print(f"Aristas en el grafo completo (g): {len(aristas_totales)}")
print(f"Aristas en el grafo final (ps): {len(aristas_finales)}")
print(f"Total de aristas excluidas (relaciones débiles y no estructurales): {len(aristas_excluidas)}")

if len(aristas_excluidas) > 0:
    # Para ver cuáles fueron, podemos convertirlas a una lista e imprimir algunas
    lista_aristas_excluidas = list(aristas_excluidas)
    print("Algunas aristas excluidas (Fuente 1, Fuente 2):")
    print(lista_aristas_excluidas[:10]) # Imprime las primeras 10

### Original

In [ ]:
archivo_excel = r"RUTA\normativa.xlsx"

# Cargar hoja "Pareadas" (equivalente a edge_list)
try:
    pareadas = pd.read_excel(archivo_excel, sheet_name="Pareadas")
    print("Datos de pareadas cargados:")
    print(pareadas.head())

    # Cargar hoja "Ponderación" (equivalente a prestaciones)
    ponderacion = pd.read_excel(archivo_excel, sheet_name="Ponderación")
    print("\nDatos de ponderación cargados:")
    print(ponderacion.head())

except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo en la ruta especificada: {archivo_excel}")
    print("Por favor, actualiza la variable 'archivo_excel' con la ruta correcta.")
    exit()


# --- PASO 2: ANALIZAR ESTRUCTURA DE DATOS ---
edge_list = pareadas.rename(columns={
    'fuente_1': 'P1',
    'fuente_2': 'P2',
    'Valor': 'weight'
}).copy()

# Verificar si hay relaciones reales entre diferentes fuentes
auto_loops = edge_list[edge_list['P1'] == edge_list['P2']]
relaciones_reales = edge_list[edge_list['P1'] != edge_list['P2']]

print(f"\nAnálisis de datos:")
print(f"Total de filas: {len(edge_list)}")
print(f"Auto-loops (fuente consigo misma): {len(auto_loops)}")
print(f"Relaciones entre diferentes fuentes: {len(relaciones_reales)}")

if len(relaciones_reales) > 0:
    print("¡Encontramos relaciones reales entre fuentes!")
    edge_list = relaciones_reales
else:
    print("Solo hay auto-loops. Creando red basada en similitudes por ministerio...")

    # Si solo hay auto-loops, crear conexiones artificiales basadas en ministerio
    # Primero, obtener la información de cada fuente
    fuentes_con_info = []

    for _, row in auto_loops.iterrows():
        fuente = row['P1']
        peso = row['weight']

        # Buscar info en ponderación
        info_fuente = ponderacion[ponderacion['fuente'] == fuente]
        if len(info_fuente) > 0:
            ministerio = info_fuente.iloc[0]['ministerio']
            ponderacion_val = info_fuente.iloc[0]['ponderación']
        else:
            ministerio = 'Sin clasificar'
            ponderacion_val = 1

        fuentes_con_info.append({
            'fuente': fuente,
            'peso_original': peso,
            'ministerio': ministerio,
            'ponderacion': ponderacion_val
        })

    # Crear DataFrame de fuentes
    df_fuentes = pd.DataFrame(fuentes_con_info)

    # Crear conexiones entre fuentes del mismo ministerio
    new_edge_list = []

    for ministerio in df_fuentes['ministerio'].unique():
        fuentes_ministerio = df_fuentes[df_fuentes['ministerio'] == ministerio]

        # Conectar todas las fuentes del mismo ministerio entre sí
        for i, fuente1 in fuentes_ministerio.iterrows():
            for j, fuente2 in fuentes_ministerio.iterrows():
                if fuente1['fuente'] != fuente2['fuente']: # Evitar auto-loops
                    # El peso es proporcional al promedio de sus pesos originales
                    peso_conexion = (fuente1['peso_original'] + fuente2['peso_original']) / 2

                    new_edge_list.append({
                        'P1': fuente1['fuente'],
                        'P2': fuente2['fuente'],
                        'weight': peso_conexion
                    })

    # Crear conexiones entre ministerios (con menor peso)
    ministerios = df_fuentes['ministerio'].unique()
    for i, min1 in enumerate(ministerios):
        for j, min2 in enumerate(ministerios):
            if i < j: # Evitar duplicados
                # Tomar las top fuentes de cada ministerio
                fuentes_min1 = df_fuentes[df_fuentes['ministerio'] == min1].nlargest(3, 'peso_original')
                fuentes_min2 = df_fuentes[df_fuentes['ministerio'] == min2].nlargest(3, 'peso_original')

                # Conectar algunas fuentes entre ministerios
                for _, f1 in fuentes_min1.iterrows():
                    for _, f2 in fuentes_min2.iterrows():
                        peso_inter_ministerial = (f1['peso_original'] + f2['peso_original']) / 4 # Peso menor
                        new_edge_list.append({
                            'P1': f1['fuente'],
                            'P2': f2['fuente'],
                            'weight': peso_inter_ministerial
                        })

    # Convertir a DataFrame
    edge_list = pd.DataFrame(new_edge_list)

    print(f"Red sintética creada: {len(edge_list)} conexiones")
    print(f"Conexiones intra-ministeriales y inter-ministeriales generadas")


if len(edge_list) > 0:
    min_w = edge_list['weight'].min()
    max_w = edge_list['weight'].max()

    if max_w != min_w:
        edge_list['weight'] = (edge_list['weight'] - min_w) / (max_w - min_w)
    else:
        edge_list['weight'] = 0.5

    # Calcular peso invertido para MST
    edge_list["inv_weight"] = 1 - edge_list['weight']

    print("\nEdge list normalizado:")
    print(edge_list.head())
else:
    print("No hay aristas válidas después de filtrar auto-loops")
    exit()


g = nx.from_pandas_edgelist(edge_list, source="P1", target="P2", edge_attr=["weight", "inv_weight"])
print(f"\n# de Nodos: {g.number_of_nodes()}")
print(f"# de Aristas: {g.number_of_edges()}")


mst = nx.minimum_spanning_tree(g, weight='inv_weight')
print(f"MST - # de Nodos: {mst.number_of_nodes()}")
print(f"MST - # de Aristas: {mst.number_of_edges()}")

ps = nx.Graph()

# Add MST ('weight' attribute only)
for u, v, w in mst.edges(data=True):
    ps.add_edge(u, v, **{'weight': w["weight"]})

# Add Edges > 0.55
umbral = 0.55
for u, v, w in g.edges(data=True):
    if w['weight'] >= umbral:
        ps.add_edge(u, v, **{'weight': w["weight"]})

print(f"\nRed final - # de Nodos: {ps.number_of_nodes()}")
print(f"Red final - # de Aristas: {ps.number_of_edges()}")

duplicados = ponderacion['fuente'].duplicated().sum()
if duplicados > 0:
    print(f"Eliminando {duplicados} fuentes duplicadas en ponderación...")
    ponderacion = ponderacion.drop_duplicates(subset=['fuente'], keep='first')

# Preparar diccionario de ponderación
ponderacion_dict = ponderacion.set_index('fuente').to_dict('index')

# Asignar atributos a cada nodo
for node_id in ps.nodes():
    if node_id in ponderacion_dict:
        node_attributes = ponderacion_dict[node_id]
        nx.set_node_attributes(ps, {node_id: node_attributes})
    else:
        # Valores por defecto para nodos sin información
        nx.set_node_attributes(ps, {node_id: {
            'ministerio': 'Sin clasificar',
            'ponderación': 1
        }})

# --- PASO 8: FUNCIONES PARA VISUALIZACIÓN ---
# Rangos para escalado
ponderaciones = ponderacion['ponderación'].dropna()
min_ponderacion = ponderaciones.min() if not ponderaciones.empty else 1
max_ponderacion = ponderaciones.max() if not ponderaciones.empty else 1

def get_node_scale_factor(node: dict):
    """Calcula factor de escala basado en ponderación."""
    ponderacion_valor = node.get('properties', {}).get('ponderación', min_ponderacion)

    if max_ponderacion == min_ponderacion:
        return 5.0

    normalized = (ponderacion_valor - min_ponderacion) / (max_ponderacion - min_ponderacion)
    min_scale = 1
    max_scale = 5
    scale = min_scale + normalized * (max_scale - min_scale)
    return scale

def get_node_label_style(node: dict):
    """Retorna estilo para etiqueta del nodo."""
    fuente_nombre = node.get('id', str(node))
    return {
        'text': fuente_nombre,
        'wrapping': 'word',
        'maximumWidth': 120,
        'textAlignment': 'center',
        'fontSize': 8
    }

# Mapeo de colores por ministerio
ministerios = ponderacion['ministerio'].unique()
colores_ministerios = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5'
]

color_map_ministerios = {
    ministerio: colores_ministerios[i % len(colores_ministerios)]
    for i, ministerio in enumerate(ministerios)
}

def get_node_color(node):
    """Obtiene color del nodo basado en ministerio."""
    ministerio = node.get('properties', {}).get('ministerio', 'Sin clasificar')
    return color_map_ministerios.get(ministerio, 'grey')

def get_edge_color(edge):
    """Color de aristas."""
    return '#666666'

# --- PASO 9: CREAR VISUALIZACIÓN CON MATPLOTLIB (MEJORADA) ---
if ps.number_of_nodes() > 0 and ps.number_of_edges() > 0:
    plt.figure(figsize=(20, 16))

    # USAR LAYOUT FORCE-DIRECTED
    pos = nx.spring_layout(ps, k=1/np.sqrt(ps.number_of_nodes()), iterations=100, seed=42)

    print(f"Usando layout spring con {ps.number_of_nodes()} nodos y {ps.number_of_edges()} aristas")

    # Preparar colores y tamaños de nodos
    node_colors = []
    node_sizes = []

    for node in ps.nodes():
        node_attrs = ps.nodes[node]
        ministerio = node_attrs.get('ministerio', 'Sin clasificar')
        color = color_map_ministerios.get(ministerio, 'grey')

        ponderacion_valor = node_attrs.get('ponderación', min_ponderacion)
        if max_ponderacion != min_ponderacion:
            normalized = (ponderacion_valor - min_ponderacion) / (max_ponderacion - min_ponderacion)
            size = 500 + normalized * 3000 # Tamaños: 500-3500
        else:
            size = 1500

        node_colors.append(color)
        node_sizes.append(size)

    # Calcular anchos de aristas basados en peso
    edge_weights = []
    for u, v, data in ps.edges(data=True):
        weight = data.get('weight', 0.5)
        width = 0.5 + weight * 3.5 # Convertir peso a ancho visual (0.5 a 4.0)
        edge_weights.append(width)

    # Dibujar la red con más detalle
    nx.draw_networkx_nodes(ps, pos, node_color=node_colors, node_size=node_sizes,
                           alpha=0.8, linewidths=1, edgecolors='black')
    nx.draw_networkx_edges(ps, pos, alpha=0.6, width=edge_weights, edge_color='#666666')
    nx.draw_networkx_labels(ps, pos, font_size=8, font_weight='bold', font_color='white')

    plt.title("Red de Fuentes Normativas\n(Tamaño = Ponderación, Color = Ministerio, Grosor = Peso de Conexión)",
              fontsize=16, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("red_fuentes_normativas.png", dpi=300, bbox_inches='tight')
    plt.show()

    print("¡Gráfico guardado como 'red_fuentes_normativas.png'!")
else:
    print("No se puede crear la visualización: la red no tiene nodos o aristas.")

# --- PASO 10: VISUALIZACIÓN INTERACTIVA (MEJORADA) ---
def mostrar_visualizacion_interactiva():
    """Función separada para mostrar la visualización interactiva"""
    try:

        if ps.number_of_nodes() > 0:
            w = GraphWidget(graph=ps)

            def get_edge_thickness(edge: dict):
                weight = edge.get('properties', {}).get('weight', 0)
                return 1 + (weight * 7)

            w.set_node_scale_factor_mapping(get_node_scale_factor)
            w.set_node_label_mapping(get_node_label_style)
            w.set_node_color_mapping(get_node_color)
            w.set_edge_color_mapping(get_edge_color)
            w.set_edge_thickness_factor_mapping(get_edge_thickness)
            w.set_sidebar(start_with='Neighborhood')

            print("\n¡Visualización interactiva creada!")
            print("Para mostrarla, ejecuta: display(w)")
            return w
        else:
            print("No se puede crear visualización interactiva: la red no tiene nodos.")
            return None

    except ImportError:
        print("\nPaquete yfiles_jupyter_graphs no disponible.")
        print("Instálalo con: pip install yfiles_jupyter_graphs")
        return None
    except Exception as e:
        print(f"\nError creando visualización interactiva: {e}")
        return None

# Crear la visualización interactiva pero no mostrarla automáticamente
w = mostrar_visualizacion_interactiva()
if w is not None:
    globals()['w'] = w 

# --- PASO 11: ESTADÍSTICAS DE LA RED ---
print(f"\n=== ESTADÍSTICAS DE LA RED ===")
print(f"Número de fuentes normativas (nodos): {ps.number_of_nodes()}")
print(f"Número de conexiones (aristas): {ps.number_of_edges()}")

if ps.number_of_nodes() > 1 and ps.number_of_edges() > 0:
    print(f"Densidad de la red: {nx.density(ps):.4f}")

    if nx.is_connected(ps):
        print(f"Diámetro de la red: {nx.diameter(ps)}")
        print(f"Longitud promedio de camino: {nx.average_shortest_path_length(ps):.4f}")
    else:
        print("La red no está completamente conectada.")
        print(f"Número de componentes conectados: {nx.number_connected_components(ps)}")

    # Nodos más centrales
    centralidad = nx.degree_centrality(ps)
    top_centrales = sorted(centralidad.items(), key=lambda x: x[1], reverse=True)[:10]

    print(f"\n=== TOP 10 FUENTES MÁS CENTRALES ===")
    for i, (fuente, centralidad_val) in enumerate(top_centrales, 1):
        node_attrs = ps.nodes[fuente]
        ministerio = node_attrs.get('ministerio', 'N/A')
        ponderacion_val = node_attrs.get('ponderación', 'N/A')
        print(f"{i:2d}. {str(fuente)[:50]:<50} | Ministerio: {ministerio} | Centralidad: {centralidad_val:.3f} | Ponderación: {ponderacion_val}")

# Distribución por ministerio
print(f"\n=== DISTRIBUCIÓN POR MINISTERIO EN LA RED ===")
ministerios_en_red = {}
for node in ps.nodes():
    ministerio = ps.nodes[node].get('ministerio', 'Sin clasificar')
    ministerios_en_red[ministerio] = ministerios_en_red.get(ministerio, 0) + 1

for ministerio, count in sorted(ministerios_en_red.items(), key=lambda x: x[1], reverse=True):
    color = color_map_ministerios.get(ministerio, 'grey')
    print(f"{ministerio}: {count} fuentes (Color: {color})")

# --- FUNCIONES AUXILIARES PARA USO POSTERIOR ---
def mostrar_con_layout(layout_tipo='spring'):
    """Función para probar diferentes layouts"""
    if ps.number_of_nodes() == 0:
        print("No hay nodos en la red")
        return

    plt.figure(figsize=(20, 16))

    if layout_tipo == 'kamada_kawai':
        pos = nx.kamada_kawai_layout(ps)
    elif layout_tipo == 'fruchterman':
        pos = nx.fruchterman_reingold_layout(ps, k=1, iterations=100)
    elif layout_tipo == 'circular':
        pos = nx.circular_layout(ps)
    else: 
        pos = nx.spring_layout(ps, k=1/np.sqrt(ps.number_of_nodes()), iterations=100, seed=42)

    node_colors = [color_map_ministerios.get(ps.nodes[node].get('ministerio', 'Sin clasificar'), 'grey') for node in ps.nodes()]
    node_sizes = []
    for node in ps.nodes():
        node_attrs = ps.nodes[node]
        ponderacion_valor = node_attrs.get('ponderación', min_ponderacion)
        if max_ponderacion != min_ponderacion:
            normalized = (ponderacion_valor - min_ponderacion) / (max_ponderacion - min_ponderacion)
            size = 500 + normalized * 3000
        else:
            size = 1500
        node_sizes.append(size)

    nx.draw_networkx_nodes(ps, pos, node_color=node_colors, node_size=node_sizes,
                           alpha=0.8, linewidths=1, edgecolors='black')
    nx.draw_networkx_edges(ps, pos, alpha=0.6, width=1, edge_color='#666666')
    nx.draw_networkx_labels(ps, pos, font_size=8, font_weight='bold', font_color='white')

    plt.title(f"Red de Fuentes Normativas - Layout: {layout_tipo}",
              fontsize=16, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    print(f"Mostrando red con layout: {layout_tipo}")

def cambiar_umbral(nuevo_umbral):
    """Función para recrear la red con diferente umbral"""
    global ps

    print(f"Recreando red con umbral: {nuevo_umbral}")
    ps_nuevo = nx.Graph()

    for u, v, w in mst.edges(data=True):
        ps_nuevo.add_edge(u, v, **{'weight': w["weight"]})

    aristas_agregadas = 0
    for u, v, w in g.edges(data=True):
        if w['weight'] >= nuevo_umbral:
            ps_nuevo.add_edge(u, v, **{'weight': w["weight"]})
            aristas_agregadas += 1

    for node_id in ps_nuevo.nodes():
        if node_id in ponderacion_dict:
            node_attributes = ponderacion_dict[node_id]
            nx.set_node_attributes(ps_nuevo, {node_id: node_attributes})
        else:
            nx.set_node_attributes(ps_nuevo, {node_id: {
                'ministerio': 'Sin clasificar',
                'ponderación': 1
            }})

    ps = ps_nuevo
    print(f"Nueva red: {ps.number_of_nodes()} nodos, {ps.number_of_edges()} aristas")
    print(f"Aristas agregadas por umbral: {aristas_agregadas}")
    mostrar_con_layout('spring')
    return ps


In [ ]:
def create_interactive_network(ps, pos, ponderacion_dict, color_map_ministerios):
    """
    Crea una visualización interactiva tipo network_plot de R usando Plotly
    """
    
    # Preparar datos de aristas
    edge_x = []
    edge_y = []
    edge_info = []
    
    for u, v, data in ps.edges(data=True):
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        
        # Información de hover para aristas
        weight = data.get('weight_normalized', 0)
        edge_info.append(f"Conexión: {u} ↔ {v}<br>Similitud: {weight:.3f}")
    
    # Crear trazado de aristas
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.8, color='rgba(125, 125, 125, 0.5)'),
        hoverinfo='none',
        mode='lines',
        showlegend=False
    )
    
    # Preparar datos de nodos por ministerio
    ministerio_traces = {}
    
    for node in ps.nodes():
        x, y = pos[node]
        node_data = ps.nodes[node]
        ministerio = node_data.get('ministerio', 'Sin clasificar')
        ponderacion = node_data.get('ponderación', 1)
        
        # Calcular tamaño del nodo
        min_pond = min([ps.nodes[n].get('ponderación', 1) for n in ps.nodes()])
        max_pond = max([ps.nodes[n].get('ponderación', 1) for n in ps.nodes()])
        
        if max_pond > min_pond:
            normalized_size = (ponderacion - min_pond) / (max_pond - min_pond)
            size = 15 + normalized_size * 40
        else:
            size = 25
            
        # Calcular centralidad
        centrality = nx.degree_centrality(ps)[node]
        
        # Información de hover
        hover_text = f"""<b>{node}</b><br>
        Ministerio: {ministerio}<br>
        Ponderación: {ponderacion}<br>
        Centralidad: {centrality:.3f}<br>
        Conexiones: {ps.degree(node)}"""
        
        # Agrupar por ministerio
        if ministerio not in ministerio_traces:
            ministerio_traces[ministerio] = {
                'x': [], 'y': [], 'text': [], 'size': [], 'hover': [], 'color': color_map_ministerios.get(ministerio, '#838D9B')
            }
        
        ministerio_traces[ministerio]['x'].append(x)
        ministerio_traces[ministerio]['y'].append(y)
        ministerio_traces[ministerio]['text'].append(node.replace(' ', '<br>'))
        ministerio_traces[ministerio]['size'].append(size)
        ministerio_traces[ministerio]['hover'].append(hover_text)
    
    # Crear trazados por ministerio (para leyenda automática)
    traces = [edge_trace]
    
    for ministerio, data in ministerio_traces.items():
        trace = go.Scatter(
            x=data['x'], y=data['y'],
            mode='markers+text',
            text=data['text'],
            textposition="middle center",
            textfont=dict(size=10, color='white', family='Arial Black'),
            hovertext=data['hover'],
            hoverinfo='text',
            name=ministerio,
            marker=dict(
                size=data['size'],
                color=data['color'],
                line=dict(width=2, color='rgba(50, 50, 50, 0.8)'),
                opacity=0.9
            ),
            showlegend=True
        )
        traces.append(trace)
    
    # Crear figura
    fig = go.Figure(data=traces)
    
    # Configurar layout
    fig.update_layout(
        title=dict(
            text='Red de Fuentes Normativas - Visualización Interactiva',
            x=0.5,
            font=dict(size=24, family='Arial', color='#1C1C1C')
        ),
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="middle",
            y=0.5,
            xanchor="left",
            x=1.02,
            font=dict(size=12),
            bgcolor="rgba(255,255,255,0.9)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1
        ),
        hovermode='closest',
        margin=dict(b=20, l=5, r=200, t=60),
        annotations=[
            dict(
                text="Tamaño = Ponderación | Hover para detalles",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.005, y=0.99,
                font=dict(color='gray', size=12)
            )
        ],
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white',
        paper_bgcolor='white',
        width=1400,
        height=900
    )
    
    return fig

# Crear y mostrar la visualización interactiva
if ps.number_of_nodes() > 0:
    print("\nCreando visualización interactiva con Plotly...")
    
    fig_interactive = create_interactive_network(ps, pos, ponderacion_dict, color_map_ministerios)
    
    # Guardar como HTML
    plot(fig_interactive, filename='red_fuentes_interactiva.html', auto_open=True)
    print("Visualización interactiva guardada como 'red_fuentes_interactiva.html'")

# Función adicional para crear gráfico con filtros dinámicos
def create_filterable_network(ps, pos, ponderacion_dict, color_map_ministerios):
    """
    Crea una versión con controles deslizantes para filtrar por ponderación
    """
    
    # Obtener rango de ponderaciones
    ponderaciones = [ps.nodes[node].get('ponderación', 1) for node in ps.nodes()]
    min_pond, max_pond = min(ponderaciones), max(ponderaciones)
    
    # Crear steps para el slider
    steps = []
    for i in range(11):  # 11 pasos (0-10)
        threshold = min_pond + (max_pond - min_pond) * i / 10
        
        # Determinar qué nodos mostrar
        visible_nodes = [node for node in ps.nodes() 
                        if ps.nodes[node].get('ponderación', 1) >= threshold]
        
        step = dict(
            method="update",
            args=[{
                "visible": [True] + [ministerio in [ps.nodes[node].get('ministerio', 'Sin clasificar') 
                                                  for node in visible_nodes] 
                                   for ministerio in color_map_ministerios.keys()]
            }],
            label=f"≥{threshold:.2f}"
        )
        steps.append(step)
    
    # Crear figura base
    fig = create_interactive_network(ps, pos, ponderacion_dict, color_map_ministerios)
    
    # Añadir slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Ponderación mínima: "},
        pad={"t": 50},
        steps=steps
    )]
    
    fig.update_layout(sliders=sliders)
    
    return fig

# Crear versión con filtros
if ps.number_of_nodes() > 0:
    print("Creando visualización con controles de filtrado...")
    
    fig_filterable = create_filterable_network(ps, pos, ponderacion_dict, color_map_ministerios)
    plot(fig_filterable, filename='red_fuentes_filtrable.html', auto_open=False)
    print("Visualización filtrable guardada como 'red_fuentes_filtrable.html'")

### Y Graphs

In [ ]:
G_test = nx.complete_graph(5)
w_test = GraphWidget(graph=G_test)
display(w_test)

In [ ]:
w = GraphWidget(graph=ps)
display(w)

In [ ]:
if 'ps2' not in locals() or ps2.number_of_nodes() == 0:
    print("Error: El objeto de grafo 'ps' no existe o está vacío.")
    print("Por favor, ejecuta el Bloque 1 primero para crear el grafo.")
else:
    
    def get_node_scale_factor(node: Dict) -> float:
        """Calcula el factor de escala del nodo basado en su 'ponderación'."""
        ponderacion_valor = node.get('properties', {}).get('ponderación', min_ponderacion)
        if max_ponderacion > min_ponderacion:
            normalized = (ponderacion_valor - min_ponderacion) / (max_ponderacion - min_ponderacion)
            return 1.0 + normalized * 4.0  
        return 2.5 

    def get_node_label_style(node: Dict) -> Dict:
        """Define el estilo y contenido de la etiqueta del nodo."""
        fuente_nombre = node.get('id', '')
        return {
            'text': fuente_nombre,
            'wrapping': 'word',
            'maximumWidth': 120,
            'textAlignment': 'center',
            'fontSize': 10
        }

    def get_node_color(node: Dict) -> str:
        """Obtiene el color del nodo basado en su ministerio."""
        ministerio = node.get('properties', {}).get('ministerio', 'Sin clasificar')
        return color_map_ministerios.get(ministerio, '#cccccc') # Usa el mapa de colores del Bloque 1

    def get_edge_thickness(edge: Dict) -> float:
        """Define el grosor de la arista basado en su 'weight'."""
        weight = edge.get('properties', {}).get('weight', 0)
        return 1.0 + (weight * 7.0) # Grosor de 1 a 8

    def get_edge_color(edge: Dict) -> str:
        """Define el color de las aristas."""
        return '#888888'

    print("\nGenerando visualización interactiva con yFiles...")
    
    w = GraphWidget(graph=ps2)

    w.set_node_scale_factor_mapping(get_node_scale_factor)
    w.set_node_color_mapping(get_node_color)
    w.set_edge_thickness_factor_mapping(get_edge_thickness)
    w.set_edge_color_mapping(get_edge_color)
    w.set_sidebar(start_with='Neighborhood')

    print("¡Visualización interactiva lista! Se mostrará a continuación.")
    display(w)

In [ ]:
w.set_node_scale_factor_mapping(get_node_scale_factor)
w.set_node_label_mapping(get_node_label_style)
w.set_node_color_mapping(get_node_color) 
w.set_edge_color_mapping(get_edge_color) 
w.set_sidebar(start_with='Neighborhood')
display(w)